# Domain Specific Language (DSL)
## Contents
1. Existing Biochemical Reaction DSL: [Catalyst.jl](https://github.com/SciML/Catalyst.jl/tree/master)
2. Custom DSL for Project

## 1. Catalyst.jl
We will now explore Catalyst.jl for specifiying biochemical reactions in human-readable format for conversion into an Ordinary Differential Equation (ODE) system.

In [ ]:
# Import Catalyst Package, Latexify (For viewing reactions)
using Catalyst, Latexify

## Review of original transcription and translation reactions:
If you recall our simple framework of reactions for transcription and translation are:

*Transcription*:
\begin{align*}
    g_i + \pi &\xrightleftharpoons[k_{1;i}^{-}]{k_{1;i}^{+}} X_i \\
    X_i &\xrightarrow{k^{\theta}_{1;i}} g_i + \pi + m_i \\
    m_i &\xrightarrow{\delta_i} \varnothing  
\end{align*}

*Translation*:
\begin{align*}
    m_i + \rho &\xrightleftharpoons[k_{2;i}^{-}]{k_{2;i}^{+}} \Xi_i \\
    \Xi_i &\xrightarrow{k^{\theta}_{2;i}} m_i + \rho + x_i \\
    x^{\omega}_i &\xrightarrow{\lambda} \varnothing  
\end{align*}

Let's look at what it looks like with the Catalyst.jl DSL!

In [ ]:

node_1 = @reaction_network begin
    (k1⁺₁, k1⁻₁), g₁ + p <--> X₁ # Gene 1 binds with RNA polymerase to form the intermediate complex in a reversible reaction
    (k1θ₁), Xᵢ --> g₁ + p + m₁ # The intermediate complex reforms gene 1 and RNAP after transcription and mRNA 1 is formed
    (δ₁), m₁ --> ∅ # mRNA 1 decays at a rate δ₁
    (k2⁺₁, k2⁻₁), m₁ + r <--> Ξ₁ # mRNA 1 binds with a ribosome to form the intermediate complex in a reversible reaction
    (k2θ₁), Ξ₁ --> m₁ + r + x₁ # The intermediate complex reforms mRNA 1 and the ribosome after translation and protein 1 is formed
    (λ), x₁ --> ∅ # Protein 1 decays at a rate λ
end

latexify(node_1)

Look how close it looks to the original reactions! But if we want to apply our resource conservation framework, we'll have to implement new code to generate the ODE system and code a new DSL.

## 2. Custom DSL For Project
To implement the mathematical model in this project, a whole back-end "engine" was developed to generate the ODE system. A DSL front-end was also made to allow you to easily enter the original reactions. Let's start with the front-end.

In [1]:
# Import necessary files
include("DSL_backend/dsl.jl")


@TFinactivate (macro with 1 method)

Now let's create a node! You can see that a different macro (@createnode) is used compared to Catalyst.jl. Also, another feature is that you can specify initial conditions very easily when you create a node unlike Catalyst! The reactions specified in the DSL are processed into a custom data structure (Node type) in the back-end.

In [2]:
# Create Node
node_1 = @createnode begin
    A ~ g = 0.5 # Node A with initial gene concentration of 0.5
    [k1⁺₁ = 4, k1⁻₁ = 1], g₁ + p <--> C₁ # gene (g₁) binds with polymerase (p) to create the intermediate complex C₁
    [k1θ₁ = 5], C --> g₁ + p + m₁ # the intermediate complex reforms the gene and polymerase while giving mRNA (m₁) after transcription
    [k2⁺₁ = 4, k2⁻₁ = 1], m₁ + r <--> X₁ # m₁ binds with a ribosome (r) to create the intermediate complex X₁
    [k2θ₁ = 5], X₁ --> m₁ + r + x₁ # The intermediate complex gives the protein (x₁) while returning the gene and ribosome
    [d₁ = 0.04], m₁ --> ∅ # mRNA decays at a rate of d = 0.04, protein dilution rate is dealt with later
end

Node(:A, 4.0, 1.0, 5.0, 4.0, 1.0, 5.0, 0.04, 0.5)

Now to specify the edges. There are a few types: activation (@activate), repression (@repress), a combination of the two (@combi), TF activation (@TFactivate), TF inactivation (@TFinactivate) and annihilation (@annihilate). 

When creating these, you can conveniently specify which the nodes that these edges connect. (e.g. A -> B means that node A activates node B)

A key feature is to the notation to specify the relations between nodes with an arrow notation to conveniently understand your system after you create it! ("->" is activation, "-!" is repression/inactivation and "-X-" is annihilation).

In [3]:
edge_1 = @activate begin
    A -> B
    [k'⁺₂ = 100, k'⁻₂ = 1], g₂ + 3x₁ <--> c₂
end

edge_2 = @repress begin
    B -! A
    [k'⁺₂ = 100, k'⁻₂ = 1], g₁ + 3x₂ <--> c
end

edge_3 = @combi begin
    A -> C, B -! C
    [k⁺ₐ₃ = 100, k⁻ₐ₃ = 1], g₃ + x₁ <--> c₃
    [k⁺ᵣ₃ = 100, k⁻ᵣ₃ = 1], g₃ + 3x₂ <--> c₃
end

edge_4 = @annihilate begin
    A -X- B
    [k⁺₁₂ = 1000, k⁻₁₂ = 0.1], m₁ + m₂ --> m∅
end

edge_5 = @TFactivate begin
    I₁ -> A
    [κ⁺₁ = 1000, κ⁺₂ = 1], x⁰₁ + 2 I₁ <--> x⁺₁
end

edge_5 = @TFinactivate begin
    I₁ -! A
    [κ⁺₁ = 1000, κ⁺₂ = 1], x⁰₁ + 2 I₁ <--> x⁺₁
end

LoadError: LoadError: UndefVarError: @annihilate not defined
in expression starting at /Users/alexandersim/Documents/Code/Y4_FYP/3_dsl.ipynb:17